# Loan Approval Status Prediction Model

# Importing neccesary libraries

In [1]:
import pandas as pd # for data handling
import matplotlib.pyplot as plt  # for visualizing the data
import seaborn as sns  # for good graphs
from sklearn.preprocessing import LabelEncoder  # to encode the categorical data
from sklearn.model_selection import train_test_split # to split the data in test and train set
from sklearn.linear_model import LogisticRegression # import the module to train
from sklearn.tree import DecisionTreeClassifier # import the second module
from sklearn.ensemble import RandomForestClassifier # import third model
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report # to check the score of both models
from sklearn.model_selection import cross_val_score # to get real score on data

# Clearing the data and pre-processing
**Reading the file**

In [2]:
df=pd.read_csv("data/data.csv")

# EDA and summarizing the data before cleaning

In [ ]:
df.head()

In [4]:
df.shape

(981, 13)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [8]:
df.duplicated().sum()

np.int64(0)

# Cleaning the data
* **Removing unwanted column and rows data to train model.**
* **367 sections are null in Loan_Status.If i dont remove this and filled it with bluff results then almost 34.7 percent data will be bluff.**

In [9]:
df=df.drop(columns=['Loan_ID','Gender'])

In [10]:
df=df.dropna(subset=['Loan_Status'])

##### **Checking no. of null data of column and working according to them**

In [ ]:
df.isnull().sum()

# Filling null columns data set
* **We have 6 'object' ,4 'float64' and 1 'int64' data type.**
* **Analyzing data for using correct function(mode,mean,median) for each column**

In [ ]:
df

* **Married/Self_Employed/Dependents/Credit_History = datatype-object, using-mode**
* **LoanAmount/Loan_Amount_Term = datatype-float64, using-median**
* **Remaining column has no null values.**

In [13]:
mode_column = ['Married','Dependents','Self_Employed','Credit_History']
median_column = ['LoanAmount','Loan_Amount_Term']

In [14]:
for column in df.columns:
    if column in mode_column:
        df[column]=df[column].fillna(df[column].mode()[0])  # we uses [0] to select only one result
    elif column in median_column:
        df[column]=df[column].fillna(df[column].median())


# Encoding the data of target
* **Using Label encoding for target(Loan_status) fisrt to plot better graphs and understand relation**
* **Keeping feature as 'object' to plot better graphs and to understand relations**

In [15]:
le = LabelEncoder()
df['Loan_Status']=le.fit_transform(df['Loan_Status'])

# EDA after data cleaning

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()
df[df.duplicated(keep=False)]

In [18]:
df = df.drop_duplicates()
df.duplicated().sum()

np.int64(0)

# Corelations between features and Loan Approval
### **Highlights :**
* **Loan is not approved for those who have credit score 0 very few is approved and most loans approved has credit score 1 so crtedit history has strong factor**
* **The approval rate is higher for married applicants compared to unmarried ones, suggesting that marital status might positively influence loan approval decisions. Similarly, applicants from semiurban areas had the highest approval rate.**
* **Income has low effect even low income also got approval but also got rejected**
* **Loan ammount, time to pay loan also has no effect on loan approval**
* **Even applicant with income 80k+ got rejected having credit score 0**

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data=df,x='ApplicantIncome',y='LoanAmount',hue='Loan_Status',
    style='Credit_History',
    palette={1: "green", 0: "red"},  # green for approved, red for rejected
    alpha=0.7
)

plt.title('Applicant Income vs Loan Amount by Loan Status')
plt.xlabel('Applicant Income')
plt.ylabel('Loan Amount (in thousands)')
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(8,6))
sns.set_context('paper')
sns.stripplot(data= df,x='Credit_History',y='ApplicantIncome',hue='Loan_Status',palette={1: "green", 0: "red"},dodge=True)
plt.title('Credit_History vs Applicant Income to check loan status')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.set_context('paper')
sns.stripplot(data= df,x='Credit_History',y='LoanAmount',hue='Loan_Status',palette={1: "green", 0: "red"},dodge=True)
plt.title('Credit_History vs Loan Ammount to check loan status')
plt.show()

In [ ]:
plt.figure(figsize=(6,5))
sns.countplot(
    data=df,
    x='Married',
    hue='Loan_Status',
    palette={1: "green", 0: "red"}
)

plt.title('Loan Status by Marital Status')
plt.xlabel('Married')
plt.ylabel('Number of Applicants')
plt.grid(axis='y', alpha=0.6)
plt.legend(title='Loan Status', labels=['Rejected', 'Approved'])
plt.show()


In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=df , x='Property_Area', hue='Loan_Status', palette={1: "green", 0: "red"})
plt.title('Property_Area by Loan_Status')
plt.ylabel('No. of Applicannts')
plt.grid(axis='y', alpha=0.6)
plt.legend(title='Loan Status',labels=['Rejected','Approved'])
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.set_context('paper')
sns.stripplot(data= df,x='Credit_History',y='Loan_Amount_Term',hue='Loan_Status',palette={1: "green", 0: "red"},dodge=True)
plt.title('Credit_History vs Applicant Income to check loan status')
plt.grid(True)
plt.show()

# Finding the outliers
* **Credit_History is clear no outliers**
* **CoapplicantIncome has values that can be realistic and partner can earn model will learn by the data**
* **For Loan_Ammount_Term loan payment time is realistic and model will learn that 12 and 480 months to pay also in option**
* **In LoanAmount min loan price is 9k(if i take it unit k) but it's not improved so it has not error data**
* #### **After exploratory data analysis and outlier inspection, certain values in ApplicantIncome and LoanAmount appeared as statistical outliers. However, upon review data of outliers, these values were found to be realistic and potentially valuable for model learning. So, I don't do any outlier trimming or capping was applied to preserve data originality.**

In [ ]:
sns.boxplot(x=df['CoapplicantIncome'])
plt.show()

In [ ]:
sns.boxplot(x=df['Loan_Amount_Term'])
plt.show()

In [ ]:
sns.boxplot(x=df['LoanAmount'])
plt.show()

In [ ]:
sns.boxplot(x=df['ApplicantIncome'])
plt.grid(True)
plt.show()

# Encoding the data of feature columns
* For other input columns(feature) one hot encoding using panda

In [29]:
ohe_column = [column for column in df.columns if df[column].dtype == 'object'] # created the list of column of type object
df=pd.get_dummies(df, columns=ohe_column, drop_first=True).astype(int)

In [30]:
df.dtypes[df.dtypes == 'object']

Series([], dtype: object)

# Splitting the data to test and train
* Here i will use 20% data to test and 80% to train

In [31]:
X = df.drop(columns=['Loan_Status'])     # X is our feature or say model input
y = df['Loan_Status']                    # y is our target or say model output
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # split the data in test and train

# Modelling
* **our data is supervised, small and classified type so i will use these two models**

# Initializing the model 1

In [32]:
log_model = LogisticRegression(max_iter=10000)
log_model.fit(X_train, y_train)
y_pred_log= log_model.predict(X_test)

In [33]:
# Logistic Regression Score
print("Logistic Regression Accuracy:", accuracy_score(y_test, log_model.predict(X_test)))
print(confusion_matrix(y_test, log_model.predict(X_test)))
print(classification_report(y_test, log_model.predict(X_test)))

Logistic Regression Accuracy: 0.7560975609756098
[[16 30]
 [ 0 77]]
              precision    recall  f1-score   support

           0       1.00      0.35      0.52        46
           1       0.72      1.00      0.84        77

    accuracy                           0.76       123
   macro avg       0.86      0.67      0.68       123
weighted avg       0.82      0.76      0.72       123



In [34]:
score_1 = cross_val_score(log_model, X_train, y_train, cv=5)
print("Cross-validation scores:", score_1)
print("Average score:", score_1.mean())


Cross-validation scores: [0.81632653 0.83673469 0.82653061 0.81632653 0.84693878]
Average score: 0.8285714285714286


# Initializing model 2

In [35]:
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)

In [36]:
# Decision Tree Score
print("Decision Tree Accuracy:", accuracy_score(y_test, tree_model.predict(X_test)))
print(confusion_matrix(y_test, tree_model.predict(X_test)))
print(classification_report(y_test, tree_model.predict(X_test)))

Decision Tree Accuracy: 0.7235772357723578
[[22 24]
 [10 67]]
              precision    recall  f1-score   support

           0       0.69      0.48      0.56        46
           1       0.74      0.87      0.80        77

    accuracy                           0.72       123
   macro avg       0.71      0.67      0.68       123
weighted avg       0.72      0.72      0.71       123



In [37]:
score_2 = cross_val_score(tree_model, X_train, y_train, cv=5)
print("Cross-validation scores:", score_2)
print("Average score:", score_2.mean())

Cross-validation scores: [0.69387755 0.68367347 0.68367347 0.70408163 0.7244898 ]
Average score: 0.6979591836734694


# Initializing model 3

In [38]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

In [39]:
# Random Forest Score
print("Random Forest Accuracy:", accuracy_score(y_test, rf.predict(X_test)))
print(confusion_matrix(y_test, rf.predict(X_test)))
print(classification_report(y_test, rf.predict(X_test)))

Random Forest Accuracy: 0.7642276422764228
[[18 28]
 [ 1 76]]
              precision    recall  f1-score   support

           0       0.95      0.39      0.55        46
           1       0.73      0.99      0.84        77

    accuracy                           0.76       123
   macro avg       0.84      0.69      0.70       123
weighted avg       0.81      0.76      0.73       123



In [40]:
score_3 = cross_val_score(rf, X_train, y_train, cv=5)
print("Cross-validation scores:", score_3)
print("Average score:", score_3.mean())

Cross-validation scores: [0.7755102  0.84693878 0.80612245 0.80612245 0.82653061]
Average score: 0.8122448979591838


# Model Evaluation
* ### Acurracy order : LogisticRegression > RandomForestClassifier > DecissionTreeClassifier
### **Key points from Output of LogisticRegression, due to higher accuracy**

* **Model perfectly finds all approved cases (recall = 1.00), but struggles to detect rejected (recall = 0.35).**
* **This means it’s biased toward predicting class 1.**
* **High precision for class 0 (1.00) means it rarely predicts class 0 incorrectly — but it predicts very rare.**
* **Out of total 123 predictions model only made 16 rejected prediction and they all preddict right, whereas 107 preddict as approved where 30 predictions was wrong.**